In [7]:
# Stop any running Ollama processes
!pkill ollama

In [8]:
# Re-install Ollama
!apt-get -q install -y curl
!curl -fsSL https://ollama.com/install.sh | sh

Reading package lists...
Building dependency tree...
Reading state information...
curl is already the newest version (7.81.0-1ubuntu1.21).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
# Start the Ollama server in the background
get_ipython().system_raw('ollama serve &')

# Give the server a moment to start up
import time
time.sleep(5)

In [10]:
# Pull the mistral model
!ollama pull mistral

# Add a small delay to observe the pull output
import time
time.sleep(3)

In [11]:
# Check if the mistral model is available to Ollama
import requests

try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        models = response.json().get("models", [])
        model_names = [model["name"] for model in models]
        if "mistral:latest" in model_names:
            print("✅ 'mistral' model found and is available to Ollama.")
        else:
            print("❌ 'mistral' model not found in Ollama's list of available models.")
            print("Available models:", model_names)
    else:
        print(f"⚠️ Could not connect to Ollama server to list models. Status code: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ Could not connect to Ollama server. Make sure it is running.")

✅ 'mistral' model found and is available to Ollama.


In [14]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 9.5 MB/s eta 0:00:00


In [16]:
import re
from pypdf import PdfReader

# ---- Step 1: Extract text from PDF ----
PDF_PATH = "legal_summaries.pdf"
reader = PdfReader(PDF_PATH)
text = "\n".join(page.extract_text() or "" for page in reader.pages)
text = re.sub(r"\s+", " ", text)
text = text.replace("Part 137 Case Summaries", "")
print("✅ PDF loaded and cleaned")

# ---- Step 2: Regex-based case splitting ----
# Case name pattern (detects "X v. Y")
case_pattern = r"(?=\b[A-Z][a-zA-Z&’'\-\. ]+ v\. [A-Z][a-zA-Z&’'\-\. ]+)"

cases = re.split(case_pattern, text)
cases = [c.strip() for c in cases if len(c.strip()) > 100]

print(f"Found roughly {len(cases)} cases.")

✅ PDF loaded and cleaned
Found roughly 184 cases.


In [20]:
cases[3]

'LLP v. Grant 2006 N.Y. Slip Op. 51005(U) (Sup. Ct., Nassau County 2006). The Nassau County Supreme Court did not apply the holding in Scordio to fee disputes arising under Part 137. Attorney argued that Part 137 did not apply, and consequently no notice was required, because the client did not actually dispute the fee, but rather simply did not pay the bills. The court held attorney’s interpretation is “untenable and would effectively eviscerate [Part 137]. The court distinguished Part 136 from Part 137, referring to the latter as wider in scope and detail and as placing the burden on the attorney to plead that the  3 dispute was not covered by Part 137 as a condition precedent to commencing an action in court.'

In [21]:
chunks = []
for i in range(0, len(cases), 4):
    chunk_text = " ".join(cases[i:i+4])
    chunks.append({"chunk_id": f"chunk_{len(chunks)}", "text": chunk_text})

print(f"✅ Created {len(chunks)} grouped chunks (~4 cases each)")

✅ Created 46 grouped chunks (~4 cases each)


In [ ]:
!pip install tqdm

In [22]:
import os, json, requests, time
from tqdm import tqdm

OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "mistral"

PROMPT_TMPL = """You are an expert legal research assistant.
You are reading excerpts from New York attorney-client fee arbitration case summaries.
Your goal is to create 2–3 RESEARCH-ORIENTED questions that would help a lawyer
or law student explore the legal issues discussed, NOT the individual cases themselves.

Guidelines:
- Focus on legal principles (e.g., rules for fee recovery, arbitration rights, filing limits).
- Do NOT include party names or case titles.
- Ask natural questions in plain English, suitable for a search assistant.
- Each question on a new line.

Case Summaries:
{text}
"""

def generate_conceptual_questions(text, model=MODEL_NAME):
    payload = {
        "model": model,
        "prompt": PROMPT_TMPL.format(text=text[:2500]),
        "stream": False
    }
    try:
        resp = requests.post(OLLAMA_URL, json=payload)
        out = resp.json().get("response", "")
        questions = [
            q.strip("•- ").strip()
            for q in out.split("\n")
            if len(q.strip()) > 6
        ]
        return questions[:3]
    except Exception as e:
        print("⚠️ Generation failed:", e)
        return []

# ---- Step 4: Generate dataset ----
eval_dataset = []
for ch in tqdm(chunks, desc="Generating research questions"):
    qs = generate_conceptual_questions(ch["text"])
    for q in qs:
        eval_dataset.append({
            "query": q,
            "relevant_chunk_ids": [ch["chunk_id"]],
            "source_preview": ch["text"][:500]
        })
    time.sleep(0.3)

# ---- Step 5: Save outputs ----
with open("legal_chunks_conceptual.json", "w") as f:
    json.dump(chunks, f, indent=2)

with open("eval_dataset_conceptual.json", "w") as f:
    json.dump(eval_dataset, f, indent=2)

print(f"\n✅ Done! Generated {len(eval_dataset)} research questions.")
print("📂 Files saved:")
print("- legal_chunks_conceptual.json")
print("- eval_dataset_conceptual.json")

with open(OUT_PATH, "w") as f:
    json.dump(generated_dataset, f, indent=2)

print(f"✅ Generated {len(generated_dataset)} questions total.")
print(f"📁 Saved dataset to: {OUT_PATH}")

Generating research questions: 100%|██████████| 46/46 [05:50<00:00,  7.62s/it]


✅ Done! Generated 138 research questions.
📂 Files saved:
- legal_chunks_conceptual.json
- eval_dataset_conceptual.json
✅ Generated 144 questions total.
📁 Saved dataset to: eval_dataset_hq.json
